In [1]:
import numpy as np

np.random.seed(42)

In [2]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Flatten, Dense
from keras.optimizers import SGD

Using TensorFlow backend.


# Data loading

In [3]:
NUM_CLASSES = 5

In [4]:
def preprocess_data(dataset):
    
    (x_train, y_train), (x_test, y_test) = dataset
    
    # NOTE: this is the shape used by Tensorflow; other backends may differ
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test  = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    x_train  = x_train.astype('float32')
    x_test   = x_test.astype('float32')
    x_train /= 255
    x_test  /= 255

    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test  = to_categorical(y_test, NUM_CLASSES)
    
    return (x_train, y_train), (x_test, y_test)

In [5]:
def load_data():
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train_public = x_train[y_train < 5]
    y_train_public = y_train[y_train < 5]
    x_test_public  = x_test[y_test < 5]
    y_test_public  = y_test[y_test < 5]
    public_dataset = (x_train_public, y_train_public), (x_test_public, y_test_public)

    x_train_private = x_train[y_train >= 5]
    y_train_private = y_train[y_train >= 5] - 5
    x_test_private  = x_test[y_test >= 5]
    y_test_private  = y_test[y_test >= 5] - 5
    private_dataset = (x_train_private, y_train_private), (x_test_private, y_test_private)
    
    return preprocess_data(public_dataset), preprocess_data(private_dataset)

# Full training

From https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py 

but see also https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

## Original

In [6]:
_, private_dataset = load_data()
(x_train, y_train), (x_test, y_test) = private_dataset

feature_layers = [
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    Activation('relu'),
    Conv2D(32, (3, 3), padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten()
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(.50),
    Dense(NUM_CLASSES),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 29404 samples, validate on 4861 samples
Epoch 1/1
29404/29404 [==============================] - 100s - loss: 0.1354 - acc: 0.9554 - val_loss: 0.0380 - val_acc: 0.9875


## Simplifying the optimizer: switching to SGD

In [7]:
_, private_dataset = load_data()
(x_train, y_train), (x_test, y_test) = private_dataset

feature_layers = [
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    Activation('relu'),
    Conv2D(32, (3, 3), padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten()
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(.50),
    Dense(NUM_CLASSES),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)

model.compile(
    loss='categorical_crossentropy', 
    optimizer=SGD(clipnorm=10000, clipvalue=10000),
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 29404 samples, validate on 4861 samples
Epoch 1/1
29404/29404 [==============================] - 107s - loss: 0.4435 - acc: 0.8463 - val_loss: 0.1595 - val_acc: 0.9488


## Getting rid of comparisons: sigmoid activations and average pooling

In [11]:
_, private_dataset = load_data()
(x_train, y_train), (x_test, y_test) = private_dataset

feature_layers = [
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    Activation('sigmoid'),
    Conv2D(32, (3, 3), padding='same'),
    Activation('sigmoid'),
    AveragePooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten()
]

classification_layers = [
    Dense(128),
    Activation('sigmoid'),
    Dropout(.50),
    Dense(NUM_CLASSES),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)

model.compile(
    loss='categorical_crossentropy', 
    optimizer=SGD(clipnorm=10000, clipvalue=10000, lr=0.1, momentum=0.9),
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 29404 samples, validate on 4861 samples
Epoch 1/25
29404/29404 [==============================] - 118s - loss: 1.6875 - acc: 0.2064 - val_loss: 1.6238 - val_acc: 0.2115
Epoch 2/25
29404/29404 [==============================] - 115s - loss: 1.6427 - acc: 0.1991 - val_loss: 1.6140 - val_acc: 0.1835
Epoch 3/25
29404/29404 [==============================] - 108s - loss: 1.6210 - acc: 0.2016 - val_loss: 1.6130 - val_acc: 0.2115
Epoch 4/25
29404/29404 [==============================] - 105s - loss: 1.6149 - acc: 0.2068 - val_loss: 1.6103 - val_acc: 0.2115
Epoch 5/25
29404/29404 [==============================] - 98s - loss: 1.6157 - acc: 0.2043 - val_loss: 1.6210 - val_acc: 0.2115
Epoch 6/25
29404/29404 [==============================] - 114s - loss: 1.6145 - acc: 0.2050 - val_loss: 1.6124 - val_acc: 0.2076
Epoch 7/25
29404/29404 [==============================] - 116s - loss: 1.6144 - acc: 0.2047 - val_loss: 1.6124 - val_acc: 0.2004
Epoch 8/25
29404/29404 [=========================

## Getting rid of logarithms: MSE loss function

In [12]:
_, private_dataset = load_data()
(x_train, y_train), (x_test, y_test) = private_dataset

feature_layers = [
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    Activation('sigmoid'),
    Conv2D(32, (3, 3), padding='same'),
    Activation('sigmoid'),
    AveragePooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten()
]

classification_layers = [
    Dense(128),
    Activation('sigmoid'),
    Dropout(.50),
    Dense(NUM_CLASSES),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)

model.compile(
    loss='mean_squared_error', 
    optimizer=SGD(clipnorm=10000, clipvalue=10000, lr=0.1, momentum=0.9), 
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 29404 samples, validate on 4861 samples
Epoch 1/25
29404/29404 [==============================] - 113s - loss: 0.1609 - acc: 0.2026 - val_loss: 0.1599 - val_acc: 0.2115
Epoch 2/25
29404/29404 [==============================] - 115s - loss: 0.1600 - acc: 0.2088 - val_loss: 0.1601 - val_acc: 0.2115
Epoch 3/25
29404/29404 [==============================] - 89s - loss: 0.1600 - acc: 0.2104 - val_loss: 0.1600 - val_acc: 0.2115
Epoch 4/25
29404/29404 [==============================] - 1493s - loss: 0.1600 - acc: 0.2102 - val_loss: 0.1600 - val_acc: 0.1971
Epoch 5/25
29404/29404 [==============================] - 101s - loss: 0.1600 - acc: 0.2101 - val_loss: 0.1599 - val_acc: 0.2076
Epoch 6/25
29404/29404 [==============================] - 87s - loss: 0.1600 - acc: 0.2081 - val_loss: 0.1600 - val_acc: 0.1971
Epoch 7/25
29404/29404 [==============================] - 85s - loss: 0.1600 - acc: 0.2064 - val_loss: 0.1599 - val_acc: 0.2115
Epoch 8/25
29404/29404 [==========================

# Transfer learning

In [10]:
public_dataset, private_dataset = load_data()

feature_layers = [
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    Activation('sigmoid'),
    Conv2D(32, (3, 3), padding='same'),
    Activation('sigmoid'),
    AveragePooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten()
]

classification_layers = [
    Dense(128),
    Activation('sigmoid'),
    Dropout(.50),
    Dense(NUM_CLASSES),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)

# pre-train on public dataset

(x_train, y_train), (x_test, y_test) = public_dataset

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

# fix lower layers

for layer in feature_layers:
    layer.trainable = False

# train on private dataset

(x_train, y_train), (x_test, y_test) = private_dataset

model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(clipnorm=10000, clipvalue=10000, lr=0.1, momentum=0.0),
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 30596 samples, validate on 5139 samples
Epoch 1/1
30596/30596 [==============================] - 127s - loss: 0.6575 - acc: 0.7377 - val_loss: 0.1155 - val_acc: 0.9677
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 46s - loss: 0.6392 - acc: 0.7765 - val_loss: 0.2877 - val_acc: 0.9128
Epoch 2/5
29404/29404 [==============================] - 45s - loss: 0.3464 - acc: 0.8867 - val_loss: 0.2041 - val_acc: 0.9358
Epoch 3/5
29404/29404 [==============================] - 46s - loss: 0.2868 - acc: 0.9088 - val_loss: 0.1712 - val_acc: 0.9467
Epoch 4/5
29404/29404 [==============================] - 56s - loss: 0.2560 - acc: 0.9178 - val_loss: 0.1528 - val_acc: 0.9529
Epoch 5/5
29404/29404 [==============================] - 49s - loss: 0.2314 - acc: 0.9265 - val_loss: 0.1550 - val_acc: 0.9490
